## How to use this Pokémon ETL Processor
The goal of this project was to offer a way to filter pokemon by types for any generation.
1. The user is prompted first if they would like the application to use either a CSV file(loaded from Kaggle) or JSON(loaded from PokéAPI).
2. In the next step, they are able to request to filter on any generation from 1 through 7 (limitation is due to the CSV not having up to generation 8).
3. Then, they are prompted on what type they specifically want to look for. (If no records show up, there may just not be any pokemon of a certain type within this generation)
4. Total base stats is calculated and then added as an additional column.
5. Afterwards, they are able to output to either a JSON or a CSV file per the user's request and the data is then loaded into a database.

The reasoning behind why there are two layers of filtering is that for the Pokémon API I was using, I was unable to get the entirety of the Pokémon list due to a limit on how many API calls I could make that was not listed in documentation.

For CSV- Kaggle dataset: https://www.kaggle.com/datasets/rounakbanik/pokemon

For JSON- PokeAPI: https://pokeapi.co/

In [1]:
import kagglehub

# Loads Pokemon CSV file
path = kagglehub.dataset_download("rounakbanik/pokemon")

print("Path to dataset files:", path)

100%|██████████| 46.6k/46.6k [00:00<00:00, 8.21MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/rounakbanik/pokemon/versions/1


In [4]:
import requests

# this portion is to get a list of types for later error handling when taking in user input
url = "https://pokeapi.co/api/v2/type/"
poke_request = requests.get(url)
types = poke_request.json()

type_names = []

for typ in types["results"]:
    type_names.append(typ["name"])

In [13]:
import pandas as pd
import sqlite3
import json
import os

# Pokémon generation ID ranges
GENERATION_RANGES = {
    '1': range(1, 152),    # Gen 1: 1-151
    '2': range(152, 253),  # Gen 2: 152-252
    '3': range(253, 387),  # Gen 3: 253-386
    '4': range(387, 494),  # Gen 4: 387-493
    '5': range(494, 650),  # Gen 5: 494-649
    '6': range(650, 722),  # Gen 6: 650-721
    '7': range(722, 810),  # Gen 7: 722-809
}

# Makes calls to the PokéAPI and returns json for a particular pokemon
def fetch_pokemon_data(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for {pokemon_id}: {response.status_code}")
        return None

# Load data from CSV
def load_data_from_csv():
    file_path = '/root/.cache/kagglehub/datasets/rounakbanik/pokemon/versions/1/pokemon.csv'
    with open(file_path, 'r') as file:
      return pd.read_csv(file_path)

# Parses base stats from API call's json response
def fetch_base_stats(pokemon_id):
    data = fetch_pokemon_data(pokemon_id)
    if data:
        name = data['name']
        types = [t['type']['name'] for t in data['types']]
        stats = {stat['stat']['name']: stat['base_stat'] for stat in data['stats']}

        # has name, types, and stats properly loaded
        return {
            'name': name,
            'types': types,
            'stats': stats
        }
    return None

# Fetch all Pokémon data for a given generation
def fetch_all_pokemon_for_generation(gen):
    pokemon_list = []
    for pokemon_id in GENERATION_RANGES[gen]:
        data = fetch_base_stats(pokemon_id)
        if data:
            pokemon_list.append({
                'id': pokemon_id,
                'name': data['name'],
                'types': data['types'],
                'generation': gen,
                'speed': data['stats'].get('speed', 0),
                'special_defense': data['stats'].get('special-defense', 0),
                'special_attack': data['stats'].get('special-attack', 0),
                'defense': data['stats'].get('defense', 0),
                'attack': data['stats'].get('attack', 0),
                'hp': data['stats'].get('hp', 0),
            })
    return pd.DataFrame(pokemon_list)

def calculate_base_total(df):
    # if is_json:
    df['base_total'] = df[['speed', 'special_defense', 'special_attack', 'defense', 'attack', 'hp']].sum(axis=1)
    # else:
    #     df['base_total'] = df[['speed', 'sp_defense', 'sp_attack', 'defense', 'attack', 'hp']].sum(axis=1)

# filters based on type input from user
def type_data(df, poke_type, is_json):
    if is_json:
        # Assuming 'types' is already a list in the DataFrame
        # No need to convert, just ensure it is treated as a list
        df['types'] = df['types'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

        # Filter the DataFrame based on poke_type for JSON
        filtered_df = df[df['types'].apply(lambda types: poke_type in types)]
    else:
        # For CSV case, check both type1 and type2
        df['types'] = df[['type1', 'type2']].apply(lambda x: [i for i in x if pd.notna(i)], axis=1)

        # Filter the DataFrame based on poke_type for CSV
        filtered_df = df[df['types'].apply(lambda types: poke_type in types)]
        filtered_df = filtered_df.drop(columns=['type1', 'type2']) # drop these as we have unified types list column now

    return filtered_df

# filters based on generation input from user
def gen_data(df, gen):
    return df[df['generation'] == int(gen)]

def save_to_output(df, output_format):
    if output_format == 'csv':
        output_file = 'output.csv'
        df.to_csv(output_file, index=False)
        print(f"Pokémon saved as a CSV file at: {output_file}.")
    elif output_format == 'json':
        output_file = 'output.json'
        df.to_json(output_file, orient='records', lines=False) # standard json format
        print(f"Pokémon saved as a JSON file at: {output_file}.")

def flatten_pokemon_data(df):
    # types is a list and needs to be turned into a string in order to be injected into sql
    df['types'] = df['types'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    return df

def load_into_sqlite(df):
    conn = sqlite3.connect('pokemon.db')
    df = flatten_pokemon_data(df)
    df.to_sql('pokemon_data', conn, if_exists='replace', index=False)
    conn.close()
    print("Pokémon data loaded into SQLite database.")

def main():
    # User input for data source
    data_source = input("Would you like to use a JSON file or a CSV file? (Enter 'json' or 'csv'): ").strip().lower()
    while data_source not in ['json', 'csv']:
        print("Invalid data source. Please reinput data source...")
        data_source = input("Would you like to use a JSON file or a CSV file? (Enter 'json' or 'csv'): ").strip().lower()
    is_json = False

    if data_source == 'json':
        # User input for generation
        is_json = True

        # TAKE EITHER GENERATION OR INPUT
        gen = input("Enter the generation to filter by (1 - 7): ").strip()
        while gen not in GENERATION_RANGES:
            print("Invalid generation. Please reinput generation...")
            gen = input("Enter the generation to filter by (1 - 7): ").strip()
        # Fetch Pokémon data from API
        df = fetch_all_pokemon_for_generation(gen)
        print(df)

    elif data_source == 'csv':
        df = load_data_from_csv()

        # only keep relevant columns for this application (such as name, gen, id, type1, type2, base_total)
        columns_to_keep = ['name', 'type1', 'type2', 'generation', 'pokedex_number', 'attack', 'defense', 'hp', 'sp_attack', 'sp_defense','speed','base_total']
        df = df[columns_to_keep]

        # renaming just to keep things unified between csv and json
        df = df.rename(columns={'pokedex_number': 'id'})
        df = df.rename(columns={'sp_attack': 'special_attack'})
        df = df.rename(columns={'sp_defense': 'special_defense'})

        # User input for filtering options
        gen = input("Enter the generation to filter by (1 - 7): ").strip()

        while gen not in GENERATION_RANGES:
            print("Invalid generation. Please reinput generation...")
            gen = input("Enter the generation to filter by (1 - 7): ").strip()
        df = gen_data(df, gen=gen)
    else:
        print("Invalid input. Exiting...")
        return

    print("\n\n")
    # filtered df by generation
    print("=== data input summary ===")
    num_records_ingestion, num_columns_ingestion = df.shape
    print(f"Number of records: {num_records_ingestion}")
    print(f"Number of columns: {num_columns_ingestion}")
    print("\n\n")

    print("Valid Pokémon types: " + ", ".join(type_names))

    # User input for Pokémon type
    poke_type = input("Enter the type to filter by (e.g., 'fire', 'water', etc.): ").strip()
    while poke_type not in type_names:
        print("Invalid type. Please reinput type...")
        poke_type = input("Enter the type to filter by (e.g., 'fire', 'water', etc.): ").strip()
    df = type_data(df, poke_type,is_json)

    print("\n\n")
    print("Calculating base total for each Pokémon...")
    calculate_base_total(df)
    # calculate_total = input("Would you like to calculate base total for each Pokémon? (yes/no): ").strip().lower()
    # if calculate_total == 'yes':
    #     calculate_base_total(df)
    # else:
    #     print("Skipping base total calculation.")

    print("\n\n")
    print("=== post processing summary ===")
    num_records_ingestion, num_columns_ingestion = df.shape
    print(f"Number of records: {num_records_ingestion}")
    print(f"Number of columns: {num_columns_ingestion}")

    # User input for output format
    output_format = input("Would you like the output in JSON or CSV format? (Enter 'json' or 'csv'): ").strip().lower()
    while output_format not in ['json', 'csv']:
        print("Invalid output format. Please reinput format...")
        output_format = input("Would you like the output in JSON or CSV format? (Enter 'json' or 'csv'): ").strip().lower()
    save_to_output(df, output_format)

    # Load into SQLite
    load_into_sqlite(df)

if __name__ == "__main__":
    main()


Would you like to use a JSON file or a CSV file? (Enter 'json' or 'csv'): json
Enter the generation to filter by (1 - 7): 7
     id         name            types generation  speed  special_defense  \
0   722       rowlet  [grass, flying]          7     42               50   
1   723      dartrix  [grass, flying]          7     52               70   
2   724    decidueye   [grass, ghost]          7     70              100   
3   725       litten           [fire]          7     70               40   
4   726     torracat           [fire]          7     90               50   
..  ...          ...              ...        ...    ...              ...   
83  805    stakataka    [rock, steel]          7     13              101   
84  806  blacephalon    [fire, ghost]          7    107               79   
85  807      zeraora       [electric]          7    143               80   
86  808       meltan          [steel]          7     34               35   
87  809     melmetal          [steel]   